In [5]:
#!pip unistall -y transformers
!pip install -U -q transformers
!pip install -q accelerate

In [2]:
## Load dolly-v2 from huggingface
from transformers import pipeline
import torch
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True)
#dolly_pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prepare prompt template for fake statement detection


In [7]:

def get_completion_dolly(statement,context):

  prompt = f"""Determine if the given statement is fake, based on the context below.
  If you don't know, say that you do not know.

  Use only information from the context to make the judgement and answer the question at the end.

  Statement:
  {statement}

  Context:
  {context}

  Response:
  """

  #print(prompt)
  #dolly_response = dolly_pipeline(prompt, max_new_tokens=50, temperature=0.2, top_p=0.3)
  dolly_response = dolly_pipeline(prompt, max_time=10, max_new_tokens=100, temperature=0.4, top_p=0.55)
  return dolly_response #[0]["generated_text"]


### Prepare the following testcases to test the above prompt and the responses from Dolly
* Will run the same prompt few times to get the probability of getting consistent results
* It takes about 4G ram and even run faster with T4 GPU on colab


### Case 1: A fake statement with authorized context found online
* Expect to flag fake


In [8]:
if False:
    statement = 'Joe Biden said not vote.'
    context = '''
    Fake Joe Biden robocall tells New Hampshire Democrats not to vote Tuesday
    The call, an apparent imitation or digital manipulation of the president's voice, says, "Voting this Tuesday only enables the Republicans in their quest to elect Donald Trump again."
    
    MANCHESTER, N.H. — The New Hampshire attorney general's office says it is investigating what appears to be an "unlawful attempt" at voter suppression after NBC News reported on a robocall impersonating President Joe Biden that told recipients not to vote in Tuesday's presidential primary.
    '''
    
    for ii in range(0,3):
      print(f'Trial {ii}')
      output = get_completion_dolly(statement, context)
      print(output[0]["generated_text"])



Trial 0
The statement is fake. The statement is an apparent imitation or digital manipulation of the president's voice.
Trial 1
The statement is fake. The statement is an apparent imitation or digital manipulation of the president's voice.
Trial 2
The statement is fake. The statement is a clear attempt at voter suppression.


### Case 2: A simple statement with the same content in context
  * Expect to flag true or I dont know


In [10]:
if False:
    statement = 'This is good.'
    context = 'This is good.'
    
    for ii in range(0,3):
      print(ii)
      output = get_completion_dolly(statement, context)
      print(output[0]["generated_text"])


0
I don't know. I don't know anything about this context.
1
I don't know. I don't know anything about this context.
2
I don't know. I don't know anything about this context.


### Case 3: A true statement with the same content in context
  * Expect to flag true

In [11]:
if False:
    statement = 'Joe Biden is the current US President.'
    context = statement
    
    
    for ii in range(0,3):
      print(ii)
      output = get_completion_dolly(statement, context)
      print(output[0]["generated_text"])


0
I don't know. I don't know anything about US Presidents.
1
I don't know. I don't know anything about US Presidents.
2
I don't know. I don't know anything about US Presidents.
